# Data Handling

__Case 1__

From https://gaia.aip.de/ 

```
SELECT TOP 50000
g.source_id, g.ra, g.dec, g.phot_g_mean_mag, g.phot_bp_mean_mag, g.phot_rp_mean_mag,
g.radius_val, g.lum_val, g.parallax, g.teff_val, g.a_g_val,
tm.j_m, tm.h_m, tm.k_m, aw.w1mpro, aw.w2mpro, aw.w3mpro,
sh.dist95, sh.dist50, sh.av95, sh.av50, sh.teff84, sh.teff50, sh.logg84, sh.logg50, sh.met84, sh.met50, sh.mass84, sh.mass50
FROM gdr2.gaia_source as g, gdr2_contrib.starhorse as sh,
catalogs.allwise as aw, catalogs.tmass as tm, gdr2.tmass_best_neighbour as tmbn, gdr2.allwise_best_neighbour as awbn
WHERE g.source_id = sh.source_id AND g.source_id = tmbn.source_id AND g.source_id = awbn.source_id
AND tmbn.tmass_oid = tm.tmass_oid AND awbn.allwise_oid = aw.allwise_oid
AND g.parallax BETWEEN 1.00 AND 28.06
```

gives a reasonable amount of data to train with; however, for our neural network, this is missing `age` and `Av`.

_Extrapolate these properties with the `isochrones` package_

In [1]:
from astropy.io import fits
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt

from isochrones import StarModel, get_ichrone
from isochrones.parsec import Parsec_Isochrone
from isochrones.mist import MIST_Isochrone

In [2]:
par =  Parsec_Isochrone()
par.use_emcee = True
print(par.bands)

mist = get_ichrone('mist')
mist.use_emcee = True
print(mist.bands)

['BP', 'G', 'H', 'J', 'K', 'RP', 'W1', 'W2', 'W3', 'g', 'i', 'r', 'z']
['B', 'BP', 'G', 'H', 'J', 'K', 'Kepler', 'RP', 'TESS', 'V', 'W1', 'W2', 'W3', 'g', 'i', 'r', 'z']


In [6]:
d = fits.open('.\\Data\\Train\\SH+phot50000.fits')[1].data
tab = Table(d)
params = dict(G=(tab['phot_g_mean_mag'], .02), BP=(tab['phot_bp_mean_mag'], 0.02), RP=(tab['phot_rp_mean_mag'], 0.02), 
             J=(tab['j_m'], 0.02), H=(tab['h_m'], 0.02), K=(tab['k_m'], 0.02), radius = (tab['radius_val'], .5), parallax = (tab['parallax'], .1))

In [4]:
sm = StarModel(par, **params, use_emcee = True)
sm.fit()
sm.samples.describe()

C:\Python36\lib\site-packages\isochrones-1.2.2.dev0-py3.6.egg\isochrones\starmodel.py:502: RuntimeWarning: divide by zero encountered in log
  lnp += np.log(self.prior('mass', masses[0]))


,age_0_0,mass_0_0,radius_0_0,logL_0_0,logg_0_0,Teff_0_0,BP_mag_0_0,G_mag_0_0,H_mag_0_0,J_mag_0_0,...,K_mag,RP_mag,W1_mag,W2_mag,W3_mag,g_mag,i_mag,r_mag,z_mag,lnprob
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,9.156792,0.818215,1.003511,-0.462183,4.460551,4767.429253,16.697130,15.997233,13.588644,14.216419,...,13.484195,15.202626,13.394501,13.431178,13.473888,17.083516,15.607965,16.005816,15.366302,-213.529775
std,0.834276,0.206826,0.627920,0.549592,0.337427,397.068099,0.210896,0.133433,0.135276,0.065065,...,0.148618,0.074317,0.154728,0.155049,0.177557,0.282016,0.078843,0.137629,0.048561,431.162645
min,6.007300,0.254895,0.260571,-1.958533,3.620097,3617.779637,15.720480,15.409053,13.256806,13.890905,...,13.002661,14.829410,12.793868,12.657631,12.620586,15.843280,15.230745,15.424165,15.055642,-5192.986613
25%,8.933014,0.702946,0.656867,-0.781004,4.415158,4540.152098,16.743562,16.029206,13.514880,14.185458,...,13.406079,15.210349,13.315175,13.354214,13.381903,17.112976,15.619068,16.033778,15.361718,-111.875771
50%,9.333097,0.737515,0.680438,-0.724166,4.641667,4633.490946,16.776903,16.047665,13.528616,14.199940,...,13.420755,15.225902,13.330266,13.375219,13.413816,17.190856,15.633411,16.053488,15.374567,-21.191297
75%,9.704367,0.813357,0.886163,-0.474445,4.657167,4869.500704,16.793856,16.058524,13.569375,14.220126,...,13.460285,15.236021,13.367909,13.416201,13.483775,17.218514,15.642829,16.066313,15.384523,-19.350758
max,10.097402,1.630833,2.889515,0.766725,5.013264,6199.111482,18.211287,17.063763,14.369372,14.723315,...,14.324896,16.007805,14.272289,14.279375,14.296799,18.859730,16.453852,17.330944,15.924999,-16.260495
